In [ ]:
!pip install ultralytics

In [ ]:
!pip install opencv-python

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8n.yaml")  # build a new model from scratch
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

# Use the model
model.train(data="coco128.yaml", epochs=3)  # train the model
metrics = model.val()  # evaluate model performance on the validation set
results = model("https://ultralytics.com/images/bus.jpg")  # predict on an image
success = model.export(format="onnx")  # export the model to ONNX format

In [2]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")
results= model("https://ultralytics.com/images/bus.jpg")


100%|██████████| 476k/476k [00:00<00:00, 5.12MB/s]
image 1/1 C:\Users\user\Documents\machine_learning\bus.jpg: 640x480 4 persons, 1 bus, 1 stop sign, 24.0ms
Speed: 1.0ms preprocess, 24.0ms inference, 7.7ms postprocess per image at shape (1, 3, 640, 640)


In [5]:
from PIL import Image
image= Image.open("bus.jpg")
results= model.predict(source=image, save=True)


0: 640x480 4 persons, 1 bus, 1 stop sign, 56.0ms
Speed: 190.3ms preprocess, 56.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict


In [1]:
!yolo mode=predict model=yolov8n.pt source=0

^C


Ultralytics YOLOv8.0.73  Python-3.9.12 torch-1.12.1 CUDA:0 (NVIDIA GeForce RTX 3090, 24576MiB)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

1/1: 0... Success  (inf frames of shape 640x480 at 30.00 FPS)

0: 480x640 (no detections), 445.0ms
0: 480x640 (no detections), 10.0ms
0: 480x640 (no detections), 10.0ms
0: 480x640 (no detections), 7.0ms
0: 480x640 (no detections), 9.0ms
0: 480x640 (no detections), 9.0ms
0: 480x640 1 clock, 9.0ms
0: 480x640 (no detections), 8.0ms
0: 480x640 (no detections), 9.0ms
0: 480x640 (no detections), 10.0ms
0: 480x640 (no detections), 7.0ms
0: 480x640 (no detections), 8.0ms
0: 480x640 (no detections), 7.0ms
0: 480x640 (no detections), 10.0ms
0: 480x640 (no detections), 8.0ms
0: 480x640 (no detections), 10.0ms
0: 480x640 (no detections), 9.0ms
0: 480x640 (no detections), 10.0ms
0: 480x640 (no detections), 9.0ms
0: 480x640 (no detections), 9.0ms
0: 480x640 (no detections), 8.0ms
0: 480x640 (no detections), 9.0ms
0: 480x640 (

In [ ]:
import cv2
#list all devices

